# Capture The Flag Shitty Add-On

>A (partial so far) solution by [Blake Burkhart](https://bburky.com)

## CTF Details

This IPython notebook is a solution for the [Capture The Flag Shitty Add-On](https://blog.wokwi.com/capture-the-flag-shitty-add-on/) by Uri Shaked. Read the blog post for full details of the CTF challenge.

For this CTF we have an ATTiny85 accessible over I²C. The full source code for the firmware is available on GitHub.

We have a number of milestones to meet:

1. [**Turn on the LED**](#Turn-on-the-LED) — Hopefully self-explanatory?
2. [**The Secret Flag**](#The-Secret-Flag) — Find a string of text in memory.
3. [**Blinking Rootkit**](#Blinking-Rootkit) — Make a the LED blink without additional I²C communication.
4. [**Replace the Flag**](#Replace-the-Flag) — Replace the secret flag string with your name.

The following resources will come in handy:

* [Blog post and CTF description](https://blog.wokwi.com/capture-the-flag-shitty-add-on/)
* [`ctf-firmware.ino` source for the CTF](https://github.com/urish/ctf-shittyaddon/blob/master/ctf-firmware/ctf-firmware.ino)
* [ATtiny8x Datasheet](https://ww1.microchip.com/downloads/en/DeviceDoc/Atmel-2586-AVR-8-bit-Microcontroller-ATtiny25-ATtiny45-ATtiny85_Datasheet.pdf)

## Setup

I tested this solution with a Raspberry Pi 2 directly connected to a Digispark clone (an ATtiny85 board). The "5v" input of the Digispark was powered from the Pi's 3.3v pin to ensure the Digsparks's outputs safely operate at 3.3v. The [`ctf-firmware-1.0.0.hex`](https://github.com/urish/ctf-shittyaddon/releases/tag/1.0.0) file was flashed to the Digispark.

The Pi must have I²C and SPI enabled in `/boot/config.txt`:

    dtparam=i2c_arm=on,i2c_arm_baudrate=10000
    dtparam=spi=on
  
I set the I²C baud rate to 10kHz to avoid errors. I think this may be due the fact that Raspberry Pis [don't support clock stretching](http://www.advamation.com/knowhow/raspberrypi/rpi-i2c-bug.html) but the [ATtiny85 and the TinyWire library may allow it](https://github.com/lucullusTheOnly/TinyWire/issues/1). In other words, the slow ATtiny85 may not send all the data fast enough with the Pi's default 100kHz baud rate, set it to something slow like 10kHz. If you don't change the baud rate you will sometimes get "OSError: [Errno 121] Remote I/O error" and may see corrupted/failed reads or writes. At first I had 40kHz working, but it quit working some reason. I found 10kHz to be completely stable, but annoyingly slow.

Also, if the AVR ISP interface is connected to the Pi's SPI pins the Digispark can be flashed directly from the Pi with `avrdude`'s linuxspi protocol:

    avrdude -v -pattiny85 -clinuxspi -b14400 -P /dev/spidev0.0 -U flash:w:ctf-firmware-1.0.0.hex 
    avrdude -v -pattiny85 -clinuxspi -b14400 -P /dev/spidev0.0 -Uefuse:w:0xfe:m -Uhfuse:w:0xdf:m -Ulfuse:w:0xe2:m
    
Don't connect the Pi to the Digispark's I²C and SPI at the same time, they share pins and things will go wrong.

Here is a photo of my Digispark connected to my Pi. The yellow and green wires connect I²C from the Pi to the Digispark. The disconnected SPI wires can be temporarily connected to reflash the Digispark. The black wire from pin 25 of the Pi is used by `avrdude` for reset. And with Wi-Fi on the Pi I can walk around with my laptop and CTF with ease.

<img src="images/wiring-photo.jpg" style="width:100%; max-width: 40em">

## I²C Read/Write

Review the source code for the CTF and you will find `onI2CReceive()` and `onI2CRequest()` handling I²C communication. During `setup()` the I²C address is configured to 0x23 and these functions are registered to run on receving and on request for I²C data.

On an I²C read, two bytes are returned by the firmware: the lower byte of the 16-bit `target` address and the byte at that adddress in memory.

When writing, the first byte is used to set lower byte of the `target` address. The upper byte is always set to zero because only one byte is being written to a `uint16_t` sized variable. The remainder of the bytes are written to memory sequentially, starting at the `target` pointer address.

A write with zero data bytes may be performed before a read to set the target address of the read.

Define a pair of `i2c_write()` and `i2c_read()` functions. These functions also track the full 16-byte target address in a `current_target` global variable.

In [1]:
from smbus2 import SMBus, i2c_msg
from io import BytesIO

CTF_I2C_ADDR = 0x23

current_target = None

def i2c_write(target, data=[]):
    global current_target

    with SMBus(1) as bus:
        msg = i2c_msg.write(CTF_I2C_ADDR, [target, *data])
        bus.i2c_rdwr(msg)
    current_target = target + len(data)

def i2c_read(target=None, count=1, ):
    global current_target

    with BytesIO() as f:
        with SMBus(1) as bus:
            # If provided, change target by writing the target address, but no data bytes
            if target is not None:
                msg = i2c_msg.write(CTF_I2C_ADDR, [target])
                bus.i2c_rdwr(msg)
                current_target = target

            for i in range(count):
                msg = i2c_msg.read(CTF_I2C_ADDR, 2)
                bus.i2c_rdwr(msg)
                target, data = msg
                f.write(bytes([data]))
                
                # If the returned target byte is ever not what we expect, raise an error
                if current_target is not None:
                    assert(target == current_target & 0xff)
                current_target += 1
    
        f.seek(0)
        return f.read()

Some quick testing. Manually set the target to 0x00 and read one byte:

In [2]:
i2c_write(0x00)
i2c_read()

b'"'

Read 4 bytes starting at 0x00:

In [3]:
i2c_read(target=0x00, count=4)

b'"\x00\x04@'

Write 6 bytes starting at 0x00. Be careful, you might corrupt memory and need to reset the ATtiny.

In [4]:
# i2c_write(0x00, b'FOOBAR')

## Turn on the LED

First, to write to an output pin the the pin must be configured as an output on AVR devices. The datasheet describes how the Port B Data Direction Register (DDRB) can be configured to use PB1 as an output. Setting DDB1 (bit 1) of DDRB enables output.

Then, writing to PORTB will change the value of the pin from low to high. 

The datasheet describes how these I/Os are accessible:

> The I/O memory space contains 64 addresses for CPU peripheral functions as Control Registers, SPI, and other I/O functions. The I/O memory can be accessed directly, or as the Data Space locations following those of the Register File, 0x20 - 0x5F.

And later describes how to access them by address:

> All ATtiny25/45/85 I/Os and peripherals are placed in the I/O space. All I/O locations may be accessed by the LD/LDS/LDD and ST/STS/STD instructions, transferring data between the 32 general purpose working registers and the I/O space. I/O Registers within the address range 0x00 - 0x1F are directly bit-accessible using the SBI and CBI instructions. In these registers, the value of single bits can be checked by using the SBIS and SBIC instructions. Refer to the instruction set section for more details. When using the I/O specific commands IN and OUT, the I/O addresses 0x00 - 0x3F must be used. When addressing I/O Registers as data space using LD and ST instructions, 0x20 must be added to these addresses. 

Great, so we can add 0x20 to the address of PORTB and DDRB to access them by memory address. Using our I²C read/write functions we can configure these I/Os in memory.

In [5]:
# The 0x20 base is called __SFR_OFFSET in avr-libc, which I used for reference
SFR_OFFSET = 0x20
# Register addresses can be found in the datasheet
PORTB_ADDR = 0x18 + SFR_OFFSET
DDRB_ADDR = 0x17 + SFR_OFFSET

# Read the current value of DDRB
ddrb, = i2c_read(DDRB_ADDR)

# Configure LED as an output pin:
# Enable DDB1 for output on PB1. This allows writing to pin 6
# of the ATtiny, the pin that the LED is connected to.
ddrb |= 1 << 1

# Set DDRB on the board
i2c_write(DDRB_ADDR, [ddrb])

# Set all output bits in PORTB to turn on the LED
i2c_write(PORTB_ADDR, [0xff])

# The actual CTF board is active-high unlike the Digispark.
# To turn the LED on on it set PORTB to 0x00.
# i2c_write(PORTB_ADDR, [0x00])

## The Secret Flag

Dump the entire flash (plus some, it loops if you read too much?) and search for the flag with a regex on the bytes.

A woefully unknown feature of Python 3's `re` module is that it supports matching on 8-bit strings (`bytes` objects). This is extremely convenient for analyzing binary data. Simply pass the pattern as `bytes` instead of a `str` to `re.search()` when matching on `bytes`.

We know the flag starts with "$FLAG:" by reviewing the source code. Dump the flash and search for a null terminated string starting with this prefix.

In [6]:
dump = i2c_read(target=0x00, count=0x10000)

In [7]:
import re

flag_match = re.search(b'\$FLAG:.*?(?=\x00)', dump)
flag = flag_match.group(0)
print(flag.decode('ascii'))

flag_address = flag_match.start()
print('Flag address:', hex(flag_address))

$FLAG:you_found_it!!! :-)
Flag address: 0x801e


## Read from High Memory Faster

It's annoying that we can't set the `target` to a full two byte `uint16_t` value and address all of memory. However, it's possible to find the upper byte of the `target` pointer in memory and set it directly. If we set it to 0x7f and start reading bytes, we should be able to read from higher memory addresses directly.

I compiled the CTF source code using [PlatformIO](https://platformio.org/) with debugging symbols enabled, and took the compiled elf and opened it with the [Ghidra](https://ghidra-sre.org/) dissasembler. After confirming that my recompilation was similar enough, I found the `target` pointer at address `mem:0064` in Ghidra by filtering the symbol list to some interesting addresses:

<img src="images/symbols.png" style="width:100%; max-width: 50em">

These addresses can also be found with `readelf`, but it's not obvious each variable is a code or data address. You have to somehow figure out that memory is at 0x800000 and code starts at zero.

```
$ readelf -s firmware.elf | grep -E 'flag|flash|target'
    21: 00800064     2 OBJECT  LOCAL  DEFAULT    3 target
    40: 00800060     2 OBJECT  LOCAL  DEFAULT    2 flash_value
    41: 0000001e    13 OBJECT  LOCAL  DEFAULT    1 _ZL4flag
    42: 00800062     2 OBJECT  LOCAL  DEFAULT    2 flash_addr
```

As a first test I tried writing a byte to address 0x64 and as expected the assert inside `i2c_read()` triggered, because the target pointer address was changed in memory. If we update `current_target` too, we can change the `target` pointer via address 0x64 and then start reading at the written address.

In [8]:
i2c_write(0x64, [0x00])
current_target = 0x00
i2c_read()

b'"'

Now, set the high byte to read from higher memory addresses. A problem is that if we set address 0x65 to 0x80 we will then start reading at 0x8066. We won't be able to control the lower byte. If we decrease the value we write to the high byte, we can just read some extra data unill we get to the desired lower byte range.

In [9]:
i2c_write(0x65, [0x7f])
current_target = 0x7f66
dump2 = i2c_read(count=0x100)

match = re.search(b'\$FLAG:.*?(?=\x00)', dump2)
print(f'Flag address {hex(flag_address)} == {hex(match.start() + 0x7f66)}')

Flag address 0x801e == 0x801e


Define a new `i2c_read_high()` function to implement this new read logic precicely:

In [10]:
TARGET_HI_ADDR = 0x65

def i2c_read_high(target, count=1):
    global current_target
    
    target_hi = target >> 8
    # Deliberately undershoot the target address by decrementing target_hi by one
    target_hi -= 1
    i2c_write(TARGET_HI_ADDR, [target_hi])
    current_target = (target_hi << 8) | (TARGET_HI_ADDR + 1)
    # Read and discard the returned bytes until we get to the desired target address
    i2c_read(count=target-current_target)
    
    return i2c_read(count=count)

# A quick test
i2c_read_high(flag_address, count=len(flag))

b'$FLAG:you_found_it!!! :-)'

# Blinking Rootkit

While there are many ways to make the LED blink, I chose to configure Timer/Counter1 to output to PB1. This way it is possible to make the LED blink without yet gaining actual code execution.

The datasheet describes how the OC1A output from Timer/Counter1 can be sent to PB1:

> **Port B, Bit 1 – MISO/AIN1/OC0B/OC1A/DO/PCINT1**
> * OC1A: Output Compare Match output: The PB1 pin can serve as an external output for the Timer/Counter1 Compare Match B when configured as an output (DDB1 set). The OC1A pin is also the output pin for the PWM mode timer function.

Read the datasheet's register description for TCCR1 for full details and then configure the TCCR1 timer. The LED will blink indefinitely without further input. If the ATtiny is reset the LED will stop blinking, this only changes the in-memory configuration of the timer and output pins.

In [11]:
TCCR1_ADDR = 0x30 + SFR_OFFSET

# Ensure DDRB is still configured to output PB1
ddrb, = i2c_read(DDRB_ADDR)
ddrb |= 0b00000010
i2c_write(DDRB_ADDR, [ddrb])

# Configure TCCR1:
#   Set bit 7 clear timer on compare match
#   Set bit 6 to 0 to leave PWM disabled
#   Set bits 5:4 to 01 to toggle PB1/OC1A (the LED)
#   Set bits 3:0 to 1111 for prescaler/16384 (as slow as possible)
tccr1 = 0b1_0_01_1111
i2c_write(TCCR1_ADDR, [tccr1])


# Replace the Flag

These will require finally figuring out whatever is acually going on in `loop()`.